In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json
from api_keys import g_key
gmaps.configure(api_key=g_key)

In [15]:
# read the cities.csv to create dataframe, previous cells are not needed to be executed as long as cities.csv was created
weather_df = pd.read_csv("output_data/weather_data.csv", encoding="utf-8")
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,Ixtapa,20.70,-105.20,80.29,83,90,8.05,1599964578,MX
1,1,Synya,65.37,58.04,42.03,93,100,9.37,1599964579,RU
2,2,Albany,42.60,-73.97,57.22,99,0,8.21,1599964375,US
3,3,Jamestown,42.10,-79.24,64.35,88,90,12.75,1599964579,US
4,4,Clyde River,70.47,-68.59,33.80,83,90,4.70,1599964580,CA


In [18]:
# Store latitude and longitude in locations
lat_lon = weather_df[['Latitude', 'Longitude']]

# Fill NaN values and convert to float
humid = weather_df["Humidity"]
#lat_lon.head()

In [19]:
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [27]:
# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 70.
ideal_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & (weather_df["Cloudiness"] == 0), :]
ideal_df = ideal_df.dropna(how='any')
ideal_df.reset_index(inplace=True)
del ideal_df['index']
ideal_df.head(10)


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,8,Mabaruma,8.20,-59.78,77.34,89,0,3.47,1599964581,GY
1,45,Acaraú,-2.89,-40.12,77.54,82,0,18.05,1599964594,BR
2,51,Geraldton,-28.77,114.60,71.60,43,0,17.22,1599964596,AU
3,65,Carnarvon,-24.87,113.63,78.80,29,0,17.22,1599964601,AU
4,115,Aksu,41.12,80.26,71.15,38,0,3.47,1599964620,CN
5,135,Kiryat Gat,31.61,34.76,73.24,9,0,0.40,1599964628,IL
6,211,Caravelas,-17.71,-39.25,70.97,86,0,6.53,1599964658,BR
7,244,Coruripe,-10.13,-36.18,73.15,79,0,11.56,1599964672,BR
8,262,Diamantino,-14.41,-56.45,79.77,25,0,6.42,1599964678,BR
9,286,São João da Barra,-21.64,-41.05,72.90,84,0,25.01,1599964686,BR


In [30]:
hotel_df = ideal_df
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""
hotel_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
0,8,Mabaruma,8.20,-59.78,77.34,89,0,3.47,1599964581,GY,
1,45,Acaraú,-2.89,-40.12,77.54,82,0,18.05,1599964594,BR,
2,51,Geraldton,-28.77,114.60,71.60,43,0,17.22,1599964596,AU,
3,65,Carnarvon,-24.87,113.63,78.80,29,0,17.22,1599964601,AU,
4,115,Aksu,41.12,80.26,71.15,38,0,3.47,1599964620,CN,


In [35]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "api_keys": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    Latitude = row["Latitude"]
    Longitude = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{Latitude},{Longitude}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#hotel_df.info()

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [39]:

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
#hotel_layer = gmaps.marker_layer(
    #locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#)


# Add layer
fig.add_layer(heat_layer)
#fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…